<div id="singlestore-header" style="display: flex; background-color: rgba(210, 255, 153, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/chart-network.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Semantic Search with OpenAI Embedding Creation</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p> This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook. </p>
    </div>
</div>

In this notebook, we will demonstrate an example of conducting semantic search on SingleStoreDB with SQL! Unlike traditional keyword-based search methods, semantic search algorithms take into account the relationships between words and their meanings, enabling them to deliver more accurate and relevant results – even when search terms are vague or ambiguous.

SingleStoreDB’s built-in parallelization and Intel SIMD-based vector processing takes care of the heavy lifting involved in processing vector data. This allows your to run your ML algorithms right in your database extremely efficiently with just 2 lines of SQL!


In this example, we use Open AI embeddings API to create embeddings for our dataset and run semantic_search using dot_product vector matching function!

## 1. Create a workspace in your workspace group

S-00 is sufficient.

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> If you have a Free Starter Workspace deployed already, select the database from drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt> to connect to that database.</p>
    </div>
</div>

## 2. Create a Database named `semantic_search`

In [1]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS semantic_search;
    %sql CREATE DATABASE semantic_search;

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Make sure to select the <tt>semantic_search</tt> database from the drop-down menu at the top of this notebook.
        It updates the <tt>connection_url</tt> which is used by the <tt>%%sql</tt> magic command and SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

## 3. Install and import required libraries

We will use the OpenAI embeddings API and will need to import the relevant dependencies accordingly.

In [2]:
!pip3 install openai==1.3.3 requests --quiet

import json
import os

from openai import OpenAI
import requests

## 4. Create an OpenAI account and get API connection details

To vectorize and embed the employee reviews and query strings, we leverage OpenAI's embeddings API. To use this API, you will need an API key, which you can get [here](https://platform.openai.com/account/api-keys). You'll need to add a payment method to actually get vector embeddings using the API, though the charges are minimal for a small example like we present here.

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>You will have to update your notebook's firewall settings to include <tt>*.*.openai.com</tt> in order to get embedddings from OpenAI APIS.</p>
    </div>
</div>

In [3]:
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass('OpenAI API Key: ')

client = OpenAI()

## 5. Create a new table in your database called reviews

In [4]:
%%sql
DROP TABLE IF EXISTS reviews;
CREATE TABLE reviews (
    date_review VARCHAR(255),
    job_title VARCHAR(255),
    location VARCHAR(255),
    review TEXT
);

## 6. Import our sample data into your table

This dataset has 15 reviews left by anonymous employees of a firm.

In [5]:
url = 'https://raw.githubusercontent.com/singlestore-labs/singlestoredb-samples/main/Tutorials/ai-powered-semantic-search/hr_sample_data.sql'

Note that we are using the `%sql` magic command here to run a query against the currently
selected database.

In [6]:
for query in [x for x in requests.get(url).text.split('\n') if x.strip()]:
     %sql {{query}}

## 7. Add vector embeddings for each review

To embed the reviews in our SingleStoreDB database, we iterate through each row in the table, make a call to OpenAI’s embeddings API with the text in the reviews field and update the new column called embeddings for each entry.

In [7]:
%sql ALTER TABLE reviews ADD embeddings BLOB;

from typing import List

reviews = %sql SELECT review FROM reviews;
reviews = [x.review for x in reviews]

def get_embeddings(inputs: List[str], model: str = 'text-embedding-ada-002') -> List[str]:
    """Return list of embeddings."""
    return [x.embedding for x in client.embeddings.create(input=inputs, model=model).data]

embeddings = get_embeddings(reviews)

for embedding, review in zip(embeddings, reviews):
    %sql UPDATE reviews SET embeddings = JSON_ARRAY_PACK('{{json.dumps(embedding)}}') WHERE review='{{review}}';

## 8. Run the semantic search algorithm with just one line of SQL

We will utilize SingleStoreDB's distributed architecture to efficiently compute the dot product of the input string (stored in searchstring) with each entry in the database and return the top 5  reviews with the highest dot product score. Each vector is normalized to length 1, hence the dot product function essentially computes the cosine similarity between two vectors – an appropriate nearness metric. SingleStoreDB makes this extremely fast because it compiles queries to machine code and runs dot_product using SIMD instructions.

In [8]:
searchstring = input('Please enter a search string: ')

search_embedding = json.dumps(get_embeddings([searchstring])[0])

results = %sql SELECT review, DOT_PRODUCT(embeddings, JSON_ARRAY_PACK('{{search_embedding}}')) AS score FROM reviews ORDER BY score DESC LIMIT 5;

print()
for i, res in enumerate(results):
    print(f'{i + 1}: {res.review} Score: {res.score}\n')

## 9. Clean up

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> If you created a new database in your Standard or Premium Workspace, you can drop the database by running the cell below. Note: this will not drop your database for Free Starter Workspaces. To drop a Free Starter Workspace, terminate the Workspace using the UI. </p>
    </div>
</div>

In [9]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS semantic_search;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>